In [ ]:
#r "nuget: FParsec, 1.1.1"

open FParsec
open System.IO
open System.Text



Installed Packages FParsec, 1.1.1

In [ ]:
// PARSERS for input

let pInt = pint32 |>> int
let pTuple = (pInt .>> pstring ",") .>>. pInt
let pLine = (pTuple .>> pstring " -> ") .>>. pTuple
let pAll = sepBy pLine newline

// HELPERS FOR CALCS
let dir a b = if a < b then 1 else (-1)
// equation of a line
let eoal x1 y1 x2 y2 x =
    let m = (y2 - y1) / (x2 - x1)
    let b = y1 - (m * x1)
    (m * x) + b

let noDiagonals (p1, p2) = fst p1 = fst p2 || snd p1 = snd p2

// Remember kids, vertical lines aren't functions
let getPoints2 = function
| ((x1, y1), (x2, y2)) when x1 = x2 -> seq { for i in y1..(dir y1 y2)..y2 do (x1, i) }
| ((x1, y1), (x2, y2)) -> seq { for i in x1..(dir x1 x2)..x2 do (i, eoal x1 y1 x2 y2 i) }

// The solver
let solve condition =
    List.filter condition
    >> List.collect (getPoints2 >> Seq.toList)
    >> List.countBy id
    >> List.filter (fun p -> snd p >= 2)
    >> List.length

let run file =
    match runParserOnFile pAll () file Encoding.ASCII with
    | Success (result, _, _) -> solve noDiagonals result, solve (fun _ -> true) result
    | _ -> failwith "oops!"
    
let day5part1solution, day5part2solution = run "day5.txt"

day5part1solution, day5part2solution


Item1,Item2
5145,16518


In [ ]:
//